In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from LogicalQ.Logical import LogicalCircuit, LogicalStatevector
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.Targets import construct_target_generic_iontrap
from LogicalQ.NoiseModel import construct_noise_model_QuantinuumH2_1
from LogicalQ.Experiments import execute_circuits
from LogicalQ.Analysis import calculate_exp_val

In [ ]:
qec_cycle_count_list = [0, 1, 2, 4, 8, 16, 32, 64]
lqc_list = []

In [ ]:
lqc_base = LogicalCircuit(1, **steane_code)
lqc_base.encode([0])

In [ ]:
for n_qec_cycles in qec_cycle_count_list:
    lqc = copy.deepcopy(lqc_base)
    for _ in range(n_qec_cycles):
        lqc.append_qec_cycle()
    lqc.measure_all()
    lqc_list.append(lqc)

lqc_list[1].draw("mpl")

In [ ]:
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
results = execute_circuits(
  lqc_list,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None,
  method="statevector", shots=1E4
)

In [ ]:
lqc_list[0].get_logical_counts(results[0].get_counts())

In [ ]:
results[0].get_counts()

In [ ]:
lsv_list = []
for result in results:
    lsv = LogicalStatevector.from_counts(result.get_counts(), n_logical_qubits=1, **steane_code)
    lsv_list.append(lsv)

In [ ]:
fig, ax = plt.subplots(dpi=128)

exp_vals = [1-lsv.logical_decomposition[0]**2 for lsv in lsv_list]

ax.bar(qec_cycle_count_list, exp_vals)

ax.set_title("Infidelity vs. Number of QEC cycles")
ax.set_xlabel("Number of QEC cycles")
ax.set_ylabel(r"$1 - P(0)$")
ax.set_yscale("log")

plt.show()